### Recuperar metadots de los modelos

In [9]:
import os
from enum import Enum
from pydantic import BaseModel
import yaml
from yaml.loader import SafeLoader
from dotenv import load_dotenv
load_dotenv('rutas_cfg.env')

class YoloVersiones(str, Enum):
    V5 = 'yolov5'
    V7 = 'yolov7'
    
class DatasetRutas(str, Enum):
    V1 = 'custom_cfg/data_v1.yaml'
    V4 = 'custom_cfg/data_v4.yaml'
    
class DatasetVersiones(str, Enum):
    V1 = 'v1'
    V4 = 'v4'

class ModelMetadata(BaseModel):
    nombre: str
    ruta_pesos: str
    yolo_ver: YoloVersiones
    dataset_ver: DatasetVersiones
    image_size: int
    
RUTA_BASE_MODELOS = os.getenv('RUTA_BASE_MODELOS')

In [10]:


lista_modelos = []
for carpeta in os.listdir(RUTA_BASE_MODELOS):
    file = open(f'{RUTA_BASE_MODELOS}/{carpeta}/opt.yaml', 'r')
    opciones_usadas = yaml.load(file, Loader=SafeLoader)
    
    yolo_ver = opciones_usadas['data'].split('/')[0]
    yolo_ver = YoloVersiones(yolo_ver)
    
    dataset_ver = opciones_usadas['data'].split('/')[1][-1]
    dataset_ver = DatasetVersiones('v' + dataset_ver)
    
    image_size = 416
    if yolo_ver == YoloVersiones.V5:
        clave_yolov5 = 'imgsz'
        image_size = opciones_usadas[clave_yolov5]
    elif yolo_ver == YoloVersiones.V7:
        clave_yolov7 = 'img_size'
        image_size = opciones_usadas[clave_yolov7][0]
        
    
    lista_modelos.append(
        ModelMetadata(
            nombre=carpeta,
            ruta_pesos=f'{RUTA_BASE_MODELOS}/{carpeta}/weights/best.pt',
            yolo_ver=yolo_ver,
            dataset_ver=dataset_ver,
            image_size=image_size
        )
    )

In [18]:
modelo = lista_modelos[0]

print(DatasetRutas[modelo.dataset_ver.name].value)

custom_cfg/data_v1.yaml
